In [7]:
!pip install -q pytube pydub mutagen

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
from pytube import YouTube
from pytube.exceptions import VideoUnavailable, RegexMatchError, AgeRestrictedError
from pydub import AudioSegment
from mutagen.easyid3 import EasyID3
from mutagen.mp3 import MP3
from IPython.display import display, clear_output
import ipywidgets as widgets

youtube_urls_list = []
output_area = widgets.Output()
drive_upload_folder = 'YouTube_MP3s'
drive_upload_folder_path = f'/content/drive/My Drive/{drive_upload_folder}'
os.makedirs(drive_upload_folder_path, exist_ok=True)
print(f"Google Drive upload folder set to: {drive_upload_folder_path}")

Google Drive upload folder set to: /content/drive/My Drive/YouTube_MP3s


In [10]:
# --- Interactive URL Input ---
url_input = widgets.Text(
    value='',
    placeholder='Paste YouTube URL here',
    description='URL:',
    style={'description_width': 'initial'}
)
add_button = widgets.Button(description='Add URL', button_style='primary', icon='plus')
done_button = widgets.Button(description='Done Adding Links', button_style='info', icon='check')

def display_url_list():
    if youtube_urls_list:
        print(f"\nCurrent list of {len(youtube_urls_list)} URLs:")
        for i, u in enumerate(youtube_urls_list, 1):
            print(f"{i}. {u}")
    else:
        print("\nNo URLs added yet.")

def on_add_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        url = url_input.value.strip()
        url_input.value = ''
        if not url:
            print("⚠️ Please enter a URL.")
            display_url_list()
            return
        if url in youtube_urls_list:
            print(f"⚠️ URL already added: {url}")
            display_url_list()
            return
        print(f"Validating: {url}...")
        try:
            yt = YouTube(url)
            _ = yt.title
            youtube_urls_list.append(url)
            print(f"✅ Added: {url}")
        except Exception as e:
            print(f"❌ Invalid or unavailable YouTube URL: {url} - {e}")
        display_url_list()

def on_done_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        if not youtube_urls_list:
            print("⚠️ No URLs were added. Processing cannot start.")
            display_url_list()
            return
        print("--- Finished Adding Links ---")
        print(f"{len(youtube_urls_list)} valid URLs collected.")
        display_url_list()
        url_input.disabled = True
        add_button.disabled = True
        done_button.disabled = True
        start_button.disabled = False
        print("\nAdjust Clip Length below and click 'Start Processing'.")

add_button.on_click(on_add_button_clicked)
done_button.on_click(on_done_button_clicked)

print("Enter YouTube URLs one by one and click 'Add URL'. Click 'Done Adding Links' when finished.")
display(url_input, add_button, done_button, output_area)

Enter YouTube URLs one by one and click 'Add URL'. Click 'Done Adding Links' when finished.


Text(value='', description='URL:', placeholder='Paste YouTube URL here', style=DescriptionStyle(description_wi…

Button(button_style='primary', description='Add URL', icon='plus', style=ButtonStyle())

Button(button_style='info', description='Done Adding Links', icon='check', style=ButtonStyle())

Output()

In [11]:
# --- Clip Length + Start Button ---
clip_length_slider = widgets.IntSlider(
    value=60, min=10, max=300, step=10,
    description='Clip Length (sec):', style={'description_width': 'initial'})
start_button = widgets.Button(description='Start Processing', button_style='success', icon='play', disabled=True)
display(clip_length_slider, start_button)

IntSlider(value=60, description='Clip Length (sec):', max=300, min=10, step=10, style=SliderStyle(description_…

Button(button_style='success', description='Start Processing', disabled=True, icon='play', style=ButtonStyle()…

In [12]:
# --- Processing Logic ---
def start_processing(b):
    for idx, url in enumerate(youtube_urls_list, 1):
        try:
            print(f"\n[{idx}/{len(youtube_urls_list)}] Processing: {url}")
            yt = YouTube(url)
            stream = yt.streams.filter(only_audio=True).first()
            temp_file = stream.download(filename=f"temp_audio_{idx}.webm")
            audio = AudioSegment.from_file(temp_file)
            trimmed = audio[:clip_length_slider.value * 1000]
            final_filename = f"{yt.title[:50].strip().replace(' ', '_')}.mp3"
            final_path = os.path.join(drive_upload_folder_path, final_filename)
            trimmed.export(final_path, format="mp3")
            os.remove(temp_file)
            audiofile = MP3(final_path, ID3=EasyID3)
            audiofile['title'] = yt.title
            audiofile['artist'] = yt.author
            audiofile['album'] = 'YouTube Batch'
            audiofile.save()
            print(f"✅ Saved: {final_path}")
        except Exception as e:
            print(f"❌ Failed to process {url}: {e}")

start_button.on_click(start_processing)